In [2]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import nltk
import string
from tqdm import tqdm
import evaluate

In [4]:
meteor = evaluate.load('meteor')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [5]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
# Dataset class
class Dataset_Translation(Dataset):
    def __init__(self, lines, word2idx_X=None, word2idx_Y=None):
        punctuation = set(string.punctuation) | set(["'", '”', '“', "’", "‘"])
        self.len = len(lines)
        self.X = []
        self.Y = []
        self.vocab_X = set()
        self.vocab_Y = set()
        for line in tqdm(lines):
            line_splited = line.split('\t')
            # Tokenize the input and target strings
            tokenized_texts = [nltk.word_tokenize(sent) for sent in line_splited[:2]]
            # Remove punctuation
            tokenized_texts = [[word for word in sent if word not in punctuation] for sent in tokenized_texts]
            self.X.append(tokenized_texts[0])
            self.Y.append(tokenized_texts[1])

            # Update the vocabulary
            self.vocab_X.update(tokenized_texts[0])
            self.vocab_Y.update(tokenized_texts[1])

        self.vocab_X.add('<s>')
        self.vocab_X.add('</s>')
        self.vocab_X.add('<PAD>')
        self.vocab_X.add('<UNK>') # DESCOMENTAR-HO

        self.vocab_Y.add('<s>')
        self.vocab_Y.add('</s>')
        self.vocab_Y.add('<PAD>')
        self.vocab_Y.add('<UNK>')

        if word2idx_X is None:
            self.word2idx_X = {word: idx for idx, word in enumerate(sorted(list(self.vocab_X)))}
        else:
            self.word2idx_X = word2idx_X

        if word2idx_Y is None:
            self.word2idx_Y = {word: idx for idx, word in enumerate(sorted(list(self.vocab_Y)))}
        else:
            self.word2idx_Y = word2idx_Y

        self.MAX_LENGTH_out = max([len(y) for y in self.Y])
        self.MAX_LENGTH_in = max([len(x) for x in self.X])


        for i in range(len(self.X)):
            self.Y[i] = ['<s>'] + self.Y[i] + ['</s>'] + ['<PAD>'] * (self.MAX_LENGTH_out - len(self.Y[i]))
            self.X[i] = ['<PAD>'] * (self.MAX_LENGTH_in - len(self.X[i])) + ['<s>'] + self.X[i] + ['</s>']

        for i in range(len(self.X)):
            self.X[i] = [self.word2idx_X[word] if word in self.word2idx_X else self.word2idx_X['<UNK>'] for word in self.X[i]]
            self.Y[i] = [self.word2idx_Y[word] if word in self.word2idx_Y else self.word2idx_Y['<UNK>'] for word in self.Y[i]]

        print(self.X)
        self.X = torch.tensor(self.X)
        self.Y = torch.tensor(self.Y)

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [16]:
with open('spa.txt', 'r') as f:
    lines = f.readlines()

from sklearn.model_selection import train_test_split

train_lines, test_lines = train_test_split(lines, test_size=0.2, random_state=42)
test_lines, val_lines = train_test_split(test_lines, test_size=0.5, random_state=42)

train_dataset = Dataset_Translation(train_lines)

100%|██████████| 102467/102467 [00:21<00:00, 4727.11it/s]
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
test_dataset = Dataset_Translation(test_lines, word2idx_X=train_dataset.word2idx_X, word2idx_Y=train_dataset.word2idx_Y)
val_dataset = Dataset_Translation(val_lines, word2idx_X=train_dataset.word2idx_X, word2idx_Y=train_dataset.word2idx_Y)

100%|██████████| 12808/12808 [00:02<00:00, 5305.46it/s]
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 12809/12809 [00:03<00:00, 3838.09it/s]


[[245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 247, 2776, 3308, 9455, 2796, 12501, 244], [245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 247, 2537, 8238, 9108, 2796, 6723, 244], [245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 247, 2217, 10202, 7617, 6723, 3417, 7654, 244], [245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 247, 1318, 7, 14051, 244], [245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 247, 2738, 9290, 14145, 4766, 14008, 8939, 244], [245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 247, 2490, 12129, 7050, 244], [245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 

In [18]:
# Dataloader class
train_dataloader = DataLoader(train_dataset, batch_size=150, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=150, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=150, shuffle=False)

In [19]:
print(train_dataset.vocab_X.__len__(), train_dataset.vocab_Y.__len__())

14167 27915


In [40]:
class Translator(nn.Module):
    def __init__(self, vocab_size_input, vocab_size_target, target_len, hidden_size=300, n_layers=1, dropout=0.1):
        super(Translator, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.target_len = target_len

        self.embedding = nn.Embedding(vocab_size_input, hidden_size)
        self.lstm_enc = nn.LSTM(hidden_size, hidden_size, n_layers, batch_first=True)
        self.lstm_dec = nn.LSTM(hidden_size, hidden_size, n_layers, batch_first=True)

        self.projection = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(hidden_size, vocab_size_target)
        )

    def forward(self, input):
        batch_size = input.size(0)
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_size).to(self.device)
        cell = torch.zeros(self.n_layers, batch_size, self.hidden_size).to(self.device)


        embedded = self.embedding(input)
        _, (hidden, cell) = self.lstm_enc(embedded, (hidden, cell))


        inp = torch.zeros(batch_size, self.hidden_size).to(self.device).unsqueeze(1)
        pred = self.projection(inp)
        for i in range(self.target_len - 1): # rm <SOS>
            out, (hidden, cell) = self.lstm_dec(inp, (hidden, cell))

            pred = torch.cat((pred, self.projection(out)), dim=1)
            inp = out

        return pred


In [46]:
model = Translator(len(train_dataset.vocab_X), len(train_dataset.vocab_Y), train_dataset.MAX_LENGTH_out+2)

model.to(model.device)

Translator(
  (embedding): Embedding(14167, 300)
  (lstm_enc): LSTM(300, 300, batch_first=True)
  (lstm_dec): LSTM(300, 300, batch_first=True)
  (projection): Sequential(
    (0): Dropout(p=0.1, inplace=False)
    (1): Linear(in_features=300, out_features=27915, bias=True)
  )
)

In [47]:
model(next(iter(train_dataloader))[0].to(model.device)).size()

torch.Size([150, 51, 27915])

In [48]:
def compute_metrics(model, dataloader):
    model.eval()
    model.target_len = dataloader.dataset.MAX_LENGTH_out+2
    special_tokens = ['<s>', '</s>', '<PAD>', '<UNK>']
    metrics = {"bleu1": 0, "bleu2": 0, "bleu3": 0, "bleu4": 0, "meteor": 0}
    word2idx_Y_inv = {idx: word for word, idx in dataloader.dataset.word2idx_Y.items()}
    counter = 0
    for inputs, targets in tqdm(dataloader):
        with torch.no_grad():
            batch_size = inputs.size(0)
            inputs = inputs.to(model.device)
            outputs = model(inputs)
            outputs = outputs.max(2)[1]

            outputs = outputs.cpu().numpy()
            targets = targets.cpu().numpy()

            for i in range(batch_size):
                outputs_i = outputs[i]
                targets_i = targets[i]

                outputs_i = [word2idx_Y_inv[idx] for idx in outputs_i]
                targets_i = [word2idx_Y_inv[idx] for idx in targets_i]

                outputs_i = [word for word in outputs_i if word not in special_tokens]
                targets_i = [word for word in targets_i if word not in special_tokens]

                targets_i = ' '.join(targets_i)
                outputs_i = ' '.join(outputs_i)

                metrics["bleu1"] += nltk.translate.bleu_score.sentence_bleu([targets_i], outputs_i, weights=(1, 0, 0, 0))
                metrics["bleu2"] += nltk.translate.bleu_score.sentence_bleu([targets_i], outputs_i, weights=(0, 1, 0, 0))
                metrics["bleu3"] += nltk.translate.bleu_score.sentence_bleu([targets_i], outputs_i, weights=(0, 0, 1, 0))
                metrics["bleu4"] += nltk.translate.bleu_score.sentence_bleu([targets_i], outputs_i, weights=(0, 0, 0, 1))
                metrics["meteor"] += meteor.compute(predictions=[outputs_i], references=[targets_i])["meteor"]
                counter += 1

    for key in metrics:
        metrics[key] /= counter
    return metrics

In [49]:
import torch.optim as optim

# Define the optimizer and loss function
optimizer = optim.AdamW(model.parameters())
criterion = nn.CrossEntropyLoss()

# Set the number of epochs
num_epochs = 10

best_val_meteor = 0
meteor_hist = []
loss_hist = []
# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()
    model.target_len = train_dataloader.dataset.MAX_LENGTH_out+2
    for inputs, targets in tqdm(train_dataloader):
        inputs = inputs.to(model.device)
        targets = targets.to(model.device)
        # Zero the gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(inputs)
        outputs = outputs.permute(0, 2, 1)
        # Compute the loss
        loss = criterion(outputs, targets)
        # Backward pass
        loss.backward()
        # Update the weights
        optimizer.step()

        # Update the running loss
        running_loss += loss.item()

    # Print the average loss for the epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_dataloader)}")
    loss_hist.append(running_loss/len(train_dataloader))

    # Evaluate on validation set
    metrics = compute_metrics(model, val_dataloader)
    print("Validation metrics:", metrics)
    meteor_hist.append(metrics["meteor"])
    if metrics["meteor"] > best_val_meteor:
        torch.save(model.state_dict(), 'best_model.pth')

model.load_state_dict(torch.load('best_model.pth'))


100%|██████████| 684/684 [08:42<00:00,  1.31it/s]


Epoch 1/10, Loss: 1.1453573542554476


  0%|          | 0/86 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use

Validation metrics: {'bleu1': 0.30348710860509753, 'bleu2': 0.14179259283074658, 'bleu3': 0.08847252922889436, 'bleu4': 0.05531207618966823, 'meteor': 0.1078555689684527}


100%|██████████| 684/684 [08:43<00:00,  1.31it/s]


...

100%|██████████| 684/684 [08:43<00:00,  1.31it/s]


Epoch 10/10, Loss: 0.38720655667851545


100%|██████████| 86/86 [01:21<00:00,  1.05it/s]


Validation metrics: {'bleu1': 0.6087533290678547, 'bleu2': 0.47015982409815815, 'bleu3': 0.4010823408757747, 'bleu4': 0.34840448228791776, 'meteor': 0.3654003091217095}


<All keys matched successfully>

In [50]:
compute_metrics(model, test_dataloader)

100%|██████████| 86/86 [01:32<00:00,  1.07s/it]


{'bleu1': 0.6208263148496942,
 'bleu2': 0.4852026491729088,
 'bleu3': 0.41766066094000265,
 'bleu4': 0.365832338343283,
 'meteor': 0.38149706465520367}

In [52]:
def show_examples(model, dataloader, num_examples):
    model.eval()
    counter_examples = 0
    special_tokens = ['<s>', '</s>', '<PAD>', '<UNK>']
    word2idx_Y_inv = {idx: word for word, idx in dataloader.dataset.word2idx_Y.items()}
    word2idx_X_inv = {idx: word for word, idx in dataloader.dataset.word2idx_X.items()}

    for inputs, targets in dataloader:
        with torch.no_grad():
            batch_size = inputs.size(0)
            inputs = inputs.to(model.device)
            outputs = model(inputs)
            outputs = outputs.max(2)[1]

            outputs = outputs.cpu().numpy()
            targets = targets.cpu().numpy()
            inputs = inputs.cpu().numpy()

            for i in range(batch_size):
                outputs_i = outputs[i]
                targets_i = targets[i]
                inputs_i = inputs[i]

                outputs_i = [word2idx_Y_inv[idx] for idx in outputs_i]
                targets_i = [word2idx_Y_inv[idx] for idx in targets_i]
                inputs_i = [word2idx_X_inv[idx] for idx in inputs_i]

                outputs_i = [word for word in outputs_i if word not in special_tokens]
                targets_i = [word for word in targets_i if word not in special_tokens]
                inputs_i = [word for word in inputs_i if word not in special_tokens]

                targets_i = ' '.join(targets_i)
                outputs_i = ' '.join(outputs_i)
                inputs_i = ' '.join(inputs_i)

                print("Original:", inputs_i)
                print("Target:", targets_i)
                print("Prediction:", outputs_i)
                print()
                counter_examples += 1
                if counter_examples == num_examples:
                    return

show_examples(model, test_dataloader, num_examples=30)

Original: How many other guys named Tom do you know
Target: ¿A cuántos otros sujetos de nombre Tom conoces
Prediction: ¿Cuántos dónde tiene Tom Tom Tom Tom

Original: I want one
Target: ¡Quiero una
Prediction: Quiero uno

Original: Tom has been convicted of drunken driving twice in the last four years
Target: Tom fue condenado por conducir en estado de ebriedad dos veces en los últimos cuatro años
Prediction: Tom ha estado estado estado los los cuatro cuatro durante durante últimos últimos años años años años

Original: Perhaps Tom wo n't go to Boston
Target: Puede que Tom no vaya a Boston
Prediction: A Tom no va a Boston Boston

Original: I 've been patient my whole life
Target: He sido paciente toda mi vida
Prediction: He he mi mi mi mi mi

Original: Some things are better left undone
Target: Algunas cosas es mejor sin hacer
Prediction: Algunos cosas cosas cosas cosas

Original: Have you understood everything
Target: ¿Has entendido todo
Prediction: ¿Lo todo todo

Original: Do you rea